# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

***

In [1]:
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10, 6)

import seaborn as sns

sns.set_context('notebook', font_scale=1.5)
sns.set_style('darkgrid', rc={'ytick.left':True, 'ytick.direction':'in',
                              'xtick.bottom':True, 'xtick.direction':'in'})

In [35]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

***

## Откройте и изучите файл

- `сalls` — количество звонков

- `minutes` — суммарная длительность звонков в минутах

- `messages` — количество sms-сообщений

- `mb_used` — израсходованный интернет-трафик в Мб

- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)


In [5]:
df = pd.read_csv('data/users_behavior.csv')

df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [6]:
df.shape

(3214, 5)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


- Пропуски отсутствуют, типы данных корректные

In [12]:
df.duplicated().sum()

0

- Дубликаты отсутствуют

## Разбейте данные на выборки

In [20]:
features = df.drop(columns=['is_ultra'])
target = df['is_ultra']

In [27]:
# Разделяю исходную выборку (60% данных на обучение, 40% на тесты и валидацию)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=.4, random_state=42
)

# Разделяю тестовую и валидационную выборку (в соотношении 1:1)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=.5, random_state=42
)

In [33]:
# Проверка
print(f'Обучающая выборка содержит {len(features_train) / len(df):.0%} исходных данных')
print(f'Валидационная выборка содержит {len(features_valid) / len(df):.0%} исходных данных')
print(f'Тестовая выборка содержит {len(features_test) / len(df):.0%} исходных данных')

Обучающая выборка содержит 60% исходных данных
Валидационная выборка содержит 20% исходных данных
Тестовая выборка содержит 20% исходных данных


## Исследуйте модели

#### Решающее дерево

In [46]:
best_dt_model = None
best_dt_accuracy = 0

for depth in range(1, 11):

    model_dt = DecisionTreeClassifier(max_depth=depth, random_state=42)
    model_dt.fit(features_train, target_train)
    predictions_valid = model_dt.predict(features_valid)

    accuracy_dt = accuracy_score(target_valid, predictions_valid)
    print(f'dt accuracy: {accuracy_dt:.0%}, depth: {depth}')

    if accuracy_dt > best_dt_accuracy:
        best_dt_model = model_dt
        best_dt_accuracy = accuracy_dt
        best_depth = depth

print('*' * 10)
print(f'best dt accuracy: {best_dt_accuracy:.0%}, best depth: {best_depth}')
print('*' * 10)

dt accuracy: 73%, depth: 1
dt accuracy: 78%, depth: 2
dt accuracy: 79%, depth: 3
dt accuracy: 78%, depth: 4
dt accuracy: 77%, depth: 5
dt accuracy: 78%, depth: 6
dt accuracy: 78%, depth: 7
dt accuracy: 80%, depth: 8
dt accuracy: 78%, depth: 9
dt accuracy: 79%, depth: 10
**********
best dt accuracy: 80%, best depth: 8
**********


/Users/mike/opt/anaconda3/envs/praktikum/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
/Users/mike/opt/anaconda3/envs/praktikum/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision

## Проверьте модель на тестовой выборке

## (бонус) Проверьте модели на адекватность

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [ ] Весь код исполняется без ошибок
- [ ] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [ ] Выполнено задание 3: проведено исследование моделей
    - [ ] Рассмотрено больше одной модели
    - [ ] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [ ] Написаны выводы по результатам исследования
- [ ] Выполнено задание 3: Проведено тестирование
- [ ] Удалось достичь accuracy не меньше 0.75
